In [1]:
from keras.callbacks import TensorBoard
%run data_gen.ipynb
%run models.ipynb

c:\users\amany\appdata\local\conda\conda\envs\gputf3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
dg = DataGen()

n_features = len(char2int)
n_steps_in = dg.max_root_len
n_steps_out = dg.max_output_len


In [3]:
batch_size = 128
n_batches = int(len(dg.words) * .7 / batch_size) 
gen = dg.gen2(batch_size=batch_size, n_batches=n_batches)

In [4]:
# define model 
# train, infenc, infdec = seq2(n_features, n_features, 64, dg.word_feat_len)
train, infenc, infdec = conv_model2(n_features, n_features, dg.word_feat_len, 64, 64)
train.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
tensorboard = TensorBoard(log_dir='logs')

Instructions for updating:
Use the retry module or similar alternatives.


In [5]:
history = train.fit_generator(gen, steps_per_epoch=n_batches, epochs = 5, callbacks=[tensorboard])

Epoch 1/5
5702/5702 [==============================] - 206s 36ms/step - loss: 0.1715 - acc: 0.9468
Epoch 2/5
5702/5702 [==============================] - 207s 36ms/step - loss: 0.0224 - acc: 0.9911
Epoch 3/5
5702/5702 [==============================] - 205s 36ms/step - loss: 0.0146 - acc: 0.9929
Epoch 4/5
5702/5702 [==============================] - 205s 36ms/step - loss: 0.0119 - acc: 0.9935
Epoch 5/5
5702/5702 [==============================] - 205s 36ms/step - loss: 0.0109 - acc: 0.9937


In [6]:
test_n_batches, test_batch_size = 30, 100 
test_gen = dg.gen2(batch_size=test_batch_size, n_batches=test_n_batches, trainset=False)

In [7]:
total, correct = 0, 0
in_word = 0
sims = []
for b in range(test_n_batches):
    [X1, X2, X3], y = next(test_gen)
    for j in range(test_batch_size):
        X33 = X3[j].reshape((1, X3.shape[1])) 
        X11 = X1[j].reshape((1, X1.shape[1], X1.shape[2], 1))
        target = predict2(infenc, infdec, X11, X33, n_steps_out, n_features)
        root = ''.join(dg.one_hot_decode(X1[j])).replace('&', ' ')
        word = ''.join(dg.one_hot_decode(y[j])).replace('&', ' ')
        targetS = ''.join(dg.one_hot_decode(target)).replace('&', ' ')
        sims.append(dg.word_sim(word, targetS))
        if dg.one_hot_decode(y[j]) == dg.one_hot_decode(target):
            correct += 1
        if root.strip() in targetS.strip():
            in_word += 1
    print(b, root, word, targetS)
    total += test_batch_size
    
word_sim_average = sum(sims)/len(sims)
print('Word Similarity Average: {0:.2f}%'.format(word_sim_average))
print('Accuracy: %.2f%%' % (float(correct)/float(total)*100.0))
print('Accuracy: %.2f%%' % (float(in_word)/float(total)*100.0))

0 matay           matayiyogaa                   matayiyogaanaageetaariidaanii
1 sal             salissiyaro                   salissiyarooroaauuu          
2 maaret          maaretaro                     maaretidaroogaanaarariiiaaana
3 yedd            yeddaanaagoo                  yeddaanaagooaaaaaaaanaanaanaa
4 cagg            cagissidori                   cagissidoriisaanii           
5 achch           achchissiyonaa                achchissiyonaaseetaakiissauuu
6 homppat         homppatiyonii                 homppatiyoniidanaadiikaariiaa
7 paall           paallogeetoo                  paallogeetoogaanaarooiiiaaari
8 baxx            baxissennee                   baxissenneetaaniiksanuukkaadi
9 warqqall        warqqalliis                   warqqalliissuanaadaasuuuuukku
10 daadir          daadira                       daadiraysuatuutuutuukuuuuuuuu
11 paatt           paatissibeokkonaa             paatissibeokkonaasiiisaakuukk
12 qoom            qoomekkee                     qoomekkeesaan